In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
import pandas as pd
import seaborn as sns
import os
import cv2
import ast

In [2]:

#! function to return a vector of frequency of all intensities from 0 to 255

def get_img_vector(img):
    img_vector = np.zeros(256)
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            img_vector[img[i][j]] += 1
    return img_vector


#! function to calculate euclidean distance between two vectors

def euclidean_distance(vector1, vector2):
    try: 
        distance = 0.0
        for i in range(len(vector1)):
            distance += np.square(vector1[i] - vector2[i])
        distance = np.sqrt(distance)
        return distance
    except Exception as e:
        print("Error")
        print(e)
        # print(i,j)
        return 0

#! function to calculate manhattan distance between two vectors
def manhattan_distance(vector1, vector2):
    return np.sum(np.abs(vector1 - vector2))

In [3]:

# functions for file handling

def formatString(string):
    string = string.replace("array([", "[")
    string = string.replace("]),array([", "],[")
    string = string.replace("])", "]")
    return string


def convertToArray(string):
    listArr = ast.literal_eval(string)
    return listArr


def tupleToList(listArr):
    array = []
    for i in range(len(listArr)):
        array.append(listArr[i])
    return array

def loadFile(path):
    with open(path, 'r') as f:
        vector = f.read()
        vector = formatString(vector)
        vector = convertToArray(vector)
        vector = tupleToList(vector)
        f.close()
    return vector

        
def saveFile(path, vector):
    with open(path, 'w') as f:
        f.write(str(vector))
        f.close()

In [4]:
def accuracy(lengthTestVectors, lengthCorrectlyClassified):
    return (lengthCorrectlyClassified/lengthTestVectors)*100

def getDegreeOfMembership(distance, gamma):

    temp = gamma * distance

    if(temp > 1):
        return 0
    elif(temp == 0):
        return 1
    else:
        return 1-temp


def getMaximumMembership(test_vector, prototypes, gamma):
    maxDegree = getDegreeOfMembership(
        euclidean_distance(test_vector, prototypes[0]), gamma)
    index = 0

    for i in range(1, len(prototypes)):
        degree = getDegreeOfMembership(
            euclidean_distance(test_vector, prototypes[i]), gamma)
        if degree > maxDegree:
            maxDegree = degree
            index = i
    return index, maxDegree


def assignClass( actinic_keratosis_membership,  basal_cell_carcinoma_membership, dermatofibroma_membership, melanoma_membership, nevus_membership, pigmented_benign_keratosis_membership, seborrheic_keratosis_membership, squamous_cell_carcinoma_membership, vascular_lesion_membership):
    maximum = max(actinic_keratosis_membership, basal_cell_carcinoma_membership, dermatofibroma_membership, melanoma_membership, nevus_membership, pigmented_benign_keratosis_membership, seborrheic_keratosis_membership, squamous_cell_carcinoma_membership, vascular_lesion_membership)
    if maximum == actinic_keratosis_membership:
        return "actinic_keratosis"
    elif maximum == basal_cell_carcinoma_membership:
        return "basal_cell_carcinoma"
    elif maximum == dermatofibroma_membership:
        return "dermatofibroma"
    elif maximum == melanoma_membership:
        return "melanoma"
    elif maximum == nevus_membership:
        return "nevus"
    elif maximum == pigmented_benign_keratosis_membership:
        return "pigmented_benign_keratosis"
    elif maximum == seborrheic_keratosis_membership:
        return "seborrheic_keratosis"
    elif maximum == squamous_cell_carcinoma_membership:
        return "squamous_cell_carcinoma"
    elif maximum == vascular_lesion_membership:
        return "vascular_lesion"
    else:
        return "none"

In [5]:

# read the test vectors

actinic_keratosis_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_actinic_keratosis_test_vectors.txt")
basal_cell_carcinoma_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_basal_cell_carcinoma_test_vectors.txt")
dermatofibroma_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_dermatofibroma_test_vectors.txt")
melanoma_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_melanoma_test_vectors.txt")
nevus_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_nevus_test_vectors.txt")
pigmented_benign_keratosis_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_pigmented_benign_keratosis_test_vectors.txt")
seborrheic_keratosis_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_seborrheic_keratosis_test_vectors.txt")
squamous_cell_carcinoma_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_squamous_cell_carcinoma_test_vectors.txt")
vascular_lesion_test_vectors = loadFile("../features/sharpened/normalised_sharpened/Test/normalised_vascular_lesion_test_vectors.txt")

In [6]:
# read the prototypes

actinic_keratosis_prototypes = loadFile('../features/sharpened/prototypes_sharpened/actinic_keratosis_prototypes_0.008.txt')
basal_cell_carcinoma_prototypes = loadFile('../features/sharpened/prototypes_sharpened/basal_cell_carcinoma_prototypes_0.008.txt')
dermatofibroma_prototypes = loadFile('../features/sharpened/prototypes_sharpened/dermatofibroma_prototypes_0.008.txt')
melanoma_prototypes = loadFile('../features/sharpened/prototypes_sharpened/melanoma_prototypes_0.05.txt')
nevus_prototypes = loadFile('../features/sharpened/prototypes_sharpened/nevus_prototypes_0.05.txt')
pigmented_benign_keratosis_prototypes = loadFile('../features/sharpened/prototypes_sharpened/pigmented_benign_keratosis_prototypes_0.008.txt')
seborrheic_keratosis_prototypes = loadFile('../features/sharpened/prototypes_sharpened/seborrheic_keratosis_prototypes_0.05.txt')
squamous_cell_carcinoma_prototypes = loadFile('../features/sharpened/prototypes_sharpened/squamous_cell_carcinoma_prototypes_0.008.txt')
vascular_lesion_prototypes = loadFile('../features/sharpened/prototypes_sharpened/vascular_lesion_prototypes_0.008.txt')

In [7]:

y_true = []
y_pred = []

# gamma = 0.0005
gamma = 0.0005

actinic_keratosis_correctly_classified = 0
basal_cell_carcinoma_correctly_classified = 0
dermatofibroma_correctly_classified = 0
melanoma_correctly_classified = 0
nevus_correctly_classified = 0
pigmented_benign_keratosis_correctly_classified = 0
seborrheic_keratosis_correctly_classified = 0
squamous_cell_carcinoma_correctly_classified = 0
vascular_lesion_correctly_classified = 0

for test_vector in actinic_keratosis_test_vectors:
    y_true.append("actinic_keratosis")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "actinic_keratosis":
        actinic_keratosis_correctly_classified += 1

for test_vector in basal_cell_carcinoma_test_vectors:
    y_true.append("basal_cell_carcinoma")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "basal_cell_carcinoma":
        basal_cell_carcinoma_correctly_classified += 1

for test_vector in dermatofibroma_test_vectors:
    y_true.append("dermatofibroma")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "dermatofibroma":
        dermatofibroma_correctly_classified += 1

for test_vector in melanoma_test_vectors:
    y_true.append("melanoma")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "melanoma":
        melanoma_correctly_classified += 1

for test_vector in nevus_test_vectors:
    y_true.append("nevus")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "nevus":
        nevus_correctly_classified += 1

for test_vector in pigmented_benign_keratosis_test_vectors:
    y_true.append("pigmented_benign_keratosis")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "pigmented_benign_keratosis":
        pigmented_benign_keratosis_correctly_classified += 1

for test_vector in seborrheic_keratosis_test_vectors:
    y_true.append("seborrheic_keratosis")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "seborrheic_keratosis":
        seborrheic_keratosis_correctly_classified += 1

for test_vector in squamous_cell_carcinoma_test_vectors:
    y_true.append("squamous_cell_carcinoma")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "squamous_cell_carcinoma":
        squamous_cell_carcinoma_correctly_classified += 1

for test_vector in vascular_lesion_test_vectors:
    y_true.append("vascular_lesion")
    actinic_keratosis_index, actinic_keratosis_degree = getMaximumMembership(test_vector, actinic_keratosis_prototypes, gamma)
    basal_cell_carcinoma_index, basal_cell_carcinoma_degree = getMaximumMembership(test_vector, basal_cell_carcinoma_prototypes, gamma)
    dermatofibroma_index, dermatofibroma_degree = getMaximumMembership(test_vector, dermatofibroma_prototypes, gamma)
    melanoma_index, melanoma_degree = getMaximumMembership(test_vector, melanoma_prototypes, gamma)
    nevus_index, nevus_degree = getMaximumMembership(test_vector, nevus_prototypes, gamma)
    pigmented_benign_keratosis_index, pigmented_benign_keratosis_degree = getMaximumMembership(test_vector, pigmented_benign_keratosis_prototypes, gamma)
    seborrheic_keratosis_index, seborrheic_keratosis_degree = getMaximumMembership(test_vector, seborrheic_keratosis_prototypes, gamma)
    squamous_cell_carcinoma_index, squamous_cell_carcinoma_degree = getMaximumMembership(test_vector, squamous_cell_carcinoma_prototypes, gamma)
    vascular_lesion_index, vascular_lesion_degree = getMaximumMembership(test_vector, vascular_lesion_prototypes, gamma)
    assigned_class = assignClass(actinic_keratosis_degree, basal_cell_carcinoma_degree, dermatofibroma_degree, melanoma_degree, nevus_degree, pigmented_benign_keratosis_degree, seborrheic_keratosis_degree, squamous_cell_carcinoma_degree, vascular_lesion_degree)
    y_pred.append(assigned_class)
    if assigned_class == "vascular_lesion":
        vascular_lesion_correctly_classified += 1

In [8]:
print("actinic_keratosis_correctly_classified: ", actinic_keratosis_correctly_classified ,"/" ,len(actinic_keratosis_test_vectors ))
print("basal_cell_carcinoma_correctly_classified: ", basal_cell_carcinoma_correctly_classified ,"/" ,len(basal_cell_carcinoma_test_vectors ))
print("dermatofibroma_correctly_classified: ", dermatofibroma_correctly_classified ,"/" ,len(dermatofibroma_test_vectors ))
print("melanoma_correctly_classified: ", melanoma_correctly_classified ,"/" ,len(melanoma_test_vectors ))
print("nevus_correctly_classified: ", nevus_correctly_classified ,"/" ,len(nevus_test_vectors ))
print("pigmented_benign_keratosis_correctly_classified: ", pigmented_benign_keratosis_correctly_classified ,"/" ,len(pigmented_benign_keratosis_test_vectors ))
print("seborrheic_keratosis_correctly_classified: ", seborrheic_keratosis_correctly_classified ,"/" ,len(seborrheic_keratosis_test_vectors ))
print("squamous_cell_carcinoma_correctly_classified: ", squamous_cell_carcinoma_correctly_classified ,"/" ,len(squamous_cell_carcinoma_test_vectors ))
print("vascular_lesion_correctly_classified: ", vascular_lesion_correctly_classified ,"/" ,len(vascular_lesion_test_vectors ))


print("y_true: ", y_true)
print("y_pred: ", y_pred)

actinic_keratosis_correctly_classified:  3 / 16
basal_cell_carcinoma_correctly_classified:  4 / 16
dermatofibroma_correctly_classified:  3 / 16
melanoma_correctly_classified:  2 / 16
nevus_correctly_classified:  11 / 16
pigmented_benign_keratosis_correctly_classified:  4 / 16
seborrheic_keratosis_correctly_classified:  2 / 3
squamous_cell_carcinoma_correctly_classified:  2 / 16
vascular_lesion_correctly_classified:  3 / 3
y_true:  ['actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'actinic_keratosis', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_carcinoma', 'basal_cell_

In [9]:

totalTestImages = len(actinic_keratosis_test_vectors) + len(basal_cell_carcinoma_test_vectors) + len(dermatofibroma_test_vectors) + len(melanoma_test_vectors) + len(nevus_test_vectors) + len(pigmented_benign_keratosis_test_vectors) + len(seborrheic_keratosis_test_vectors) + len(squamous_cell_carcinoma_test_vectors) + len(vascular_lesion_test_vectors)
print("totalTestImages: ", totalTestImages)

totalCorrectlyClassified =  actinic_keratosis_correctly_classified + basal_cell_carcinoma_correctly_classified + dermatofibroma_correctly_classified + melanoma_correctly_classified + nevus_correctly_classified + pigmented_benign_keratosis_correctly_classified + seborrheic_keratosis_correctly_classified + squamous_cell_carcinoma_correctly_classified + vascular_lesion_correctly_classified
print("totalCorrectlyClassified: ", totalCorrectlyClassified)

accuracy = totalCorrectlyClassified / totalTestImages * 100
print("accuracy: ", accuracy)

totalTestImages:  118
totalCorrectlyClassified:  34
accuracy:  28.8135593220339
